# Build A Regression Model with Keras - Part B

**Normalize the data (5 marks)**

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?

# Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <font size="3">
        <ol>
            <li><a href="#section-1">Download and Clean Dataset</a></li>
            <li><a href="#section-2">Split data</a></li>
            <li><a href="#section-3">Import Keras</a></li>
            <li><a href="#section-4">Build the Neural Network</a></li>
            <li><a href="#section-5">Train the Network</a></li>
            <li><a href="#section-6">Model Evaluation</a></li>
            <li><a href="#section-7">Create List of MSE</a></li>
        </ol>
    </font>
</div>

## Download and Clean Dataset <a id="section-1"></a>


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented. 
# If you run this notebook on a different environment, e.g. your desktop, you may need to uncomment and install certain libraries.

#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6
#!pip install scikit-learn==0.24.2

In [2]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [3]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [4]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


## Split data <a id="section-2"></a>

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Normalize the data by substracting the mean and dividing by the standard deviation:

In [8]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()


Let's randomly split the data into a training and test sets by holding 30% of the data for testing.

In [9]:
from sklearn.model_selection import train_test_split

predictors_train, predictors_test, target_train, target_test = train_test_split(predictors_norm, target, test_size = 0.3)

Save the number of predictors to *n_cols* since we will need this number when building our network.

In [10]:

n_cols = predictors_norm.shape[1] # number of predictors
print(n_cols)

8


## Import Keras <a id="section-3"></a>

#### Let's go ahead and import the Keras library


In [11]:
import keras

Using TensorFlow backend.


As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [12]:
from keras.models import Sequential
from keras.layers import Dense

## Build the Neural Network <a id="section-4"></a>

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [13]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has tone hidden layer of 10 hidden units and a ReLU activation function.

## Train the Network <a id="section-5"></a>

Let's call the function now to creare our model

In [14]:
# build the model
model = regression_model()

Next, we will train the model using the *fit* method. We will train the model for 50 epochs.

In [15]:
# fit the model
model.fit(predictors_norm, target, epochs=50) 



Epoch 1/50





1030/1030 [==============================] - 0s 167us/step - loss: 1561.7140
Epoch 2/50
1030/1030 [==============================] - 0s 32us/step - loss: 1539.1479
Epoch 3/50
1030/1030 [==============================] - 0s 31us/step - loss: 1516.2213
Epoch 4/50
1030/1030 [==============================] - 0s 32us/step - loss: 1492.7606
Epoch 5/50
1030/1030 [==============================] - 0s 30us/step - loss: 1468.0542
Epoch 6/50
1030/1030 [==============================] - 0s 29us/step - loss: 1442.0971
Epoch 7/50
1030/1030 [==============================] - 0s 29us/step - loss: 1414.4103
Epoch 8/50
1030/1030 [==============================] - 0s 31us/step - loss: 1384.9687
Epoch 9/50
1030/1030 [==============================] - 0s 31us/step - loss: 1354.0201
Epoch 10/50
1030/1030 [==============================] - 0s 30us/step - loss: 1321.0831
Epoch 11/50
1030/1030 [==============================] - 0s 29us/step - loss: 1285.4283
Epoch 12/50
1030/1030 [==========

## Model Evaluation <a id="section-6"></a>

Now we evaluate the model on the test data using the *predict* method.

In [16]:
predicted_targets = model.predict(predictors_test)

We now compute the mean squared error between the predicted concrete strength target and the actual concrete strength target by using the *train_test_split* function from Scikit-learn.

In [17]:
from sklearn.metrics import mean_squared_error

# Convert pandas Series to numpy array
target_test_array = target_test.values

# Compute mean squared error
mse = mean_squared_error(target_test_array, predicted_targets)
print("The mean squared error is: {}".format(np.around(mse, decimals = 2)))

The mean squared error is: 190.7


## Create List of MSE <a id="section-7"></a>

Now we repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [18]:
# Initialize empty MSE list
MSEs = []

Repeat the steps 50 times and save MSE in list:

In [19]:

# build the model
model = regression_model()
for i in range(50):
    # fit the model
    model.fit(predictors_norm, target, epochs=50) 
    predicted_targets = model.predict(predictors_test)
    MSEs.append(mean_squared_error(target_test_array, predicted_targets))

Epoch 1/50
1030/1030 [==============================] - 0s 192us/step - loss: 1585.6774
Epoch 2/50
1030/1030 [==============================] - 0s 44us/step - loss: 1558.5986
Epoch 3/50
1030/1030 [==============================] - 0s 36us/step - loss: 1530.9009
Epoch 4/50
1030/1030 [==============================] - 0s 33us/step - loss: 1502.0651
Epoch 5/50
1030/1030 [==============================] - 0s 32us/step - loss: 1471.8955
Epoch 6/50
1030/1030 [==============================] - 0s 31us/step - loss: 1440.0096
Epoch 7/50
1030/1030 [==============================] - 0s 31us/step - loss: 1406.4950
Epoch 8/50
1030/1030 [==============================] - 0s 31us/step - loss: 1371.4481
Epoch 9/50
1030/1030 [==============================] - 0s 33us/step - loss: 1334.4292
Epoch 10/50
1030/1030 [==============================] - 0s 32us/step - loss: 1295.3696
Epoch 11/50
1030/1030 [==============================] - 0s 33us/step - loss: 1254.9125
Epoch 12/50
1030/1030 [=================

We now report the mean and standard deviation of the mean squared errors:

In [20]:
print("The mean of the mean squared errors is: {}".format(np.around(np.mean(MSEs), decimals = 1)))
print("The standard deviation of the mean squared errors is: {}".format(np.around(np.std(MSEs), decimals = 1)))

The mean of the mean squared errors is: 34.5
The standard deviation of the mean squared errors is: 30.7


As we can see the mean is lower compared to that from Part A